# Collect Data About Heng Chun Weather

A web crawler to collect data from CWB 

collect Heng Chun weather

https://www.cwb.gov.tw/V8/E/W/OBS_Station.html?ID=46759

In [52]:
# importing modules

from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import pandas as pd
from datetime import datetime

In [65]:
# Collect data
# Analysis CSS
url = "https://www.cwb.gov.tw/V8/E/W/Observe/MOD/24hr/46759.html?T=698"
html = urlopen(url).read()
soup = BeautifulSoup(html, "lxml")
print(soup.prettify())#show html


<html>
 <body>
  <tr data-countyid="10013" data-cstname="恆春" data-estname="Hengchun">
   <th class="is_show" headers="time" scope="row">
    05/10
    <br class="visible-md"/>
    12:30
   </th>
   <td class="is_show" headers="temp">
    <span class="tem-C is-active">
     32.5
    </span>
    <span class="tem-F is-hidden">
     91
    </span>
   </td>
   <td class="is_show" headers="weather">
    <img alt="CLEAR" src="/V8/assets/img/weather_icons/weathers/svg_icon/day/01.svg" title="CLEAR"/>
   </td>
   <td headers="w-1">
    <span class="wind">
     SW
    </span>
   </td>
   <td headers="w-2">
    <span class="wind_2 is-active">
     2.0
    </span>
    <span class="wind_1 is-hidden">
     2
    </span>
   </td>
   <td headers="w-3">
    <span class="wind_2 is-active">
     -
    </span>
    <span class="wind_1 is-hidden">
     -
    </span>
   </td>
   <td headers="visible-1">
    &gt;30
   </td>
   <td headers="hum">
    55
   </td>
   <td headers="pre">
    1010.2
   </td>
   <td

In [66]:
#check time 
time_log = soup.find_all('th')
data_time = [i.get_text() for i in time_log]
print(data_time[:5])#show head 5 data  

['05/10 12:30', '05/10 12:20', '05/10 12:10', '05/10 12:00', '05/10 11:50']


In [136]:
# lack of time year
# Add year
now_year = str(datetime.now().year) + '/'
data_time_year = [now_year + i for i in data_time]
data = pd.DataFrame({'time':data_time_year})
print(data.head())#show head 5 data  

               time
0  2021/05/10 12:30
1  2021/05/10 12:20
2  2021/05/10 12:10
3  2021/05/10 12:00
4  2021/05/10 11:50


In [132]:
soup_td = soup.find_all('td')
headers_set = set([i['headers'][0] for i in soup_td])
headers_list = list(headers_set)
headers_list.sort()
print(headers_list)#show headers


['hum', 'pre', 'rain', 'sunlight', 'temp', 'visible-1', 'w-1', 'w-2', 'w-3', 'weather']


In [139]:
for i in headers_list:
    data_value = soup.find_all('td', headers=i)
    data[i] = [j.get_text() for j in data_value]

print(data.head())#show data head

               time hum     pre rain sunlight    temp visible-1  w-1   w-2  \
0  2021/05/10 12:30  55  1010.2  0.0      6.5  32.591       >30   SW  2.02   
1  2021/05/10 12:20  55  1010.3  0.0      6.3  32.290       >30   SW  2.32   
2  2021/05/10 12:10  55  1010.4  0.0      6.2  32.090       >30  WSW  2.12   
3  2021/05/10 12:00  52  1010.6  0.0      6.0  32.791       >30  WSW  1.51   
4  2021/05/10 11:50  56  1010.7  0.0      5.8  31.989       >30  WSW  1.82   

    w-3 weather  
0    --          
1    --          
2    --          
3  6.34          
4    --          


In [151]:
# lose weather data
# look weather data
weather_data = soup.find_all('td', headers='weather')
print(weather_data[0])#analysis weather

<td class="is_show" headers="weather"><img alt="CLEAR" src="/V8/assets/img/weather_icons/weathers/svg_icon/day/01.svg" title="CLEAR"/></td>


In [159]:
# The target is img alt=?.
data['weather'] = [i.img['alt'] for i in weather_data]
print(data)

                 time hum     pre rain sunlight    temp visible-1  w-1   w-2  \
0    2021/05/10 12:30  55  1010.2  0.0      6.5  32.591       >30   SW  2.02   
1    2021/05/10 12:20  55  1010.3  0.0      6.3  32.290       >30   SW  2.32   
2    2021/05/10 12:10  55  1010.4  0.0      6.2  32.090       >30  WSW  2.12   
3    2021/05/10 12:00  52  1010.6  0.0      6.0  32.791       >30  WSW  1.51   
4    2021/05/10 11:50  56  1010.7  0.0      5.8  31.989       >30  WSW  1.82   
..                ...  ..     ...  ...      ...     ...       ...  ...   ...   
145  2021/05/09 12:20  61  1009.8  0.0      4.2  32.290       >30  WNW  2.22   
146  2021/05/09 12:10  60  1009.9  0.0      4.1  32.390       >30   NW  1.82   
147  2021/05/09 12:00  62  1010.0  0.0      4.0  31.889       >30   NW  2.12   
148  2021/05/09 11:50  62  1010.0  0.0      3.8  32.190       >30  WNW  2.42   
149  2021/05/09 11:40  58  1010.1  0.0      3.7  32.791       >30    W  2.42   

      w-3 weather  
0      --   CLEAR  

In [160]:
data.to_csv('data\\CWB_Heng_Chun .csv', encoding='utf-8-sig')